In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV

from sklearn.compose import ColumnTransformer

from sklearn.inspection import permutation_importance

from sklearn.metrics import make_scorer, roc_auc_score
import os
%matplotlib inline

In [3]:
def score_func(y, y_pred):
    score = roc_auc_score(y, y_pred, multi_class="ovr")
    return score

my_scores = make_scorer(score_func=score_func, greater_is_better=True, needs_proba=True, needs_threshold=False)

In [4]:
os.getcwd()

'/Users/zhengyuanrui/Decoding_SALT/Decode_SALT/1_Script/2_model/5_Valence012_pre'

In [5]:
os.chdir("../../../3_Result/Valence012_pre/1_Logistic/2_Trials-Back")

In [6]:
os.getcwd()

'/Users/zhengyuanrui/Decoding_SALT/Decode_SALT/3_Result/Valence012_pre/1_Logistic/2_Trials-Back'

In [7]:
df_noref = pd.read_csv("/Users/zhengyuanrui/Decoding_SALT/Decode_SALT/2_Data/df_no_ref012.csv")
df_selfref = pd.read_csv("/Users/zhengyuanrui/Decoding_SALT/Decode_SALT/2_Data/df_self_ref012.csv")

In [8]:
df_noref.head()

,Subject,ExpNo,BlockNo,TrialNo,TrialNo1b,TrialNo2b,TrialNo3b,TrialNo4b,ACC,RT,...,Valence2b,ACC3b,RT3b,ismatch3b,Valence3b,ACC4b,RT4b,ismatch4b,Valence4b,label
0,1001,Exp1a,1,16,15,14,13,12,1,1065,...,1,1,1049,1,0,1,865,0,0,2
1,1001,Exp1a,1,36,35,34,33,32,1,929,...,2,1,913,0,0,1,633,1,0,2
2,1001,Exp1a,1,49,48,47,46,45,0,880,...,0,1,865,0,2,1,592,1,0,1
3,1001,Exp1a,1,50,49,48,47,46,0,888,...,0,1,648,1,0,1,865,0,2,2
4,1001,Exp1a,1,51,50,49,48,47,0,777,...,1,1,776,0,0,1,648,1,0,0


In [9]:
df_selfref.head()

,Subject,ExpNo,BlockNo,TrialNo,TrialNo1b,TrialNo2b,TrialNo3b,TrialNo4b,Identity,ACC,...,Valence2b,ACC3b,RT3b,ismatch3b,Valence3b,ACC4b,RT4b,ismatch4b,Valence4b,label
0,3010,Exp3a,1,5,4,3,2,1,Self,1,...,2,0,660,1,2,1,822,1,1,1
1,3010,Exp3a,1,6,5,4,3,2,Other,1,...,1,0,608,1,2,0,660,1,2,2
2,3010,Exp3a,1,7,6,5,4,3,Self,1,...,1,1,747,1,1,0,608,1,2,0
3,3010,Exp3a,1,8,7,6,5,4,Other,0,...,2,1,657,0,1,1,747,1,1,0
4,3010,Exp3a,1,9,8,7,6,5,Self,1,...,0,1,631,0,2,1,657,0,1,0


In [10]:
X_norefb = df_noref.iloc[:, 11:-1].values
X_selfrefb = df_selfref.iloc[:, 12:-1].values


y_noref = df_noref["label"].values
y_selfref = df_selfref["label"].values

norefcolb = ["RT1b", "RT2b", "RT3b", "RT4b", "ACC1b", "ismatch1b", "Valence1b", "ACC2b", "ismatch2b", "Valence2b", "ACC3b", "ismatch3b", "Valence3b", "ACC4b", "ismatch4b", "Valence4b"]
selfrefcolb = ["RT1b", "RT2b", "RT3b", "RT4b", "ACC1b", "ismatch1b", "Valence1b", "ACC2b", "ismatch2b", "Valence2b", "ACC3b", "ismatch3b", "Valence3b", "ACC4b", "ismatch4b", "Valence4b"]

In [11]:
X_selfrefb.shape

(108263, 16)

In [12]:
y_noref.shape

(92824,)

In [13]:
y_selfref.shape

(108263,)

In [14]:
groups_no = df_noref["Subject"].values
groups_self = df_selfref["Subject"].values

In [15]:
logo = LeaveOneGroupOut()

In [16]:
def lr_within_task(X, y, group, source):
    df_result = dict(subID=[], score=[], source=[], target=[])# source拟合的，target预测的condition
    feature_importance = []
    feature_coef = []
    
    for train, test in logo.split(X, y, groups=group):
        test_sub = np.unique(group[test])[0]
        df_result["subID"].append(test_sub)

        ct = ColumnTransformer(transformers=[("rt", StandardScaler(), [1, 5, 9, 13])],
                       remainder='passthrough')
        logi = make_pipeline(
            ct, 
            LogisticRegressionCV(Cs = np.logspace(-6, 3, 7), cv = 5, class_weight='balanced', 
                                 random_state=123, max_iter=5000, multi_class="ovr"))
        


        logi.fit(X=X[train], y=y[train])
        feature_coef.append(logi.steps[-1][-1].coef_)
        im = permutation_importance(logi, X[test], y[test], scoring=my_scores, n_repeats=20, n_jobs=-1, random_state=123)
        feature_importance.append(im['importances_mean'])
        y_pred = logi.predict_proba(X[test])
        score = roc_auc_score(y[test], y_pred, multi_class='ovr')

        df_result['score'].append(score)
        df_result['source'].append(source)
        df_result['target'].append(source)

    return pd.DataFrame(df_result), feature_importance, feature_coef


In [17]:
def lr_cross_task(X_source, y_source, X_target, y_target, target_group, source_name, target_name):
    df_result = dict(subID=[], score=[], source=[], target=[])# source拟合的，target预测的condition
    feature_importance = []
    feature_coef = []
    ct = ColumnTransformer(transformers=[("rt", StandardScaler(), [1, 5, 9, 13])],
                       remainder='passthrough')
    logi = make_pipeline(
            ct, 
            LogisticRegressionCV(Cs = np.logspace(-6, 3, 7), cv = 5, class_weight='balanced', 
                                 random_state=123, max_iter=5000, multi_class="ovr"))



    logi.fit(X=X_source, y=y_source)


    for sub in np.unique(target_group):
        idx_sub = target_group == sub
        feature_sub = X_target[idx_sub]
        label_sub = y_target[idx_sub]



        im = permutation_importance(logi, feature_sub, label_sub, scoring=my_scores, n_repeats=20, n_jobs=-1, random_state=123)
        feature_importance.append(im['importances_mean'])
        feature_coef.append(logi.steps[-1][-1].coef_)
        y_pred = logi.predict_proba(feature_sub)
        score = roc_auc_score(label_sub, y_pred, multi_class="ovr")

        df_result['subID'].append(sub)
        df_result["score"].append(score)
        df_result["source"].append(source_name)
        df_result["target"].append(target_name)


    return pd.DataFrame(df_result), feature_importance, feature_coef

In [17]:
score_no, im_no, coef_no = lr_within_task(X = X_norefb, y = y_noref, group = groups_no, source="No_Ref")
score_self, im_self, coef_self = lr_within_task(X = X_selfrefb, y = y_selfref, group = groups_self, source="Self_Ref")

In [18]:
df_im_no = pd.DataFrame(np.array(im_no), columns=norefcolb)
df_im_no

,RT1b,RT2b,RT3b,RT4b,ACC1b,ismatch1b,Valence1b,ACC2b,ismatch2b,Valence2b,ACC3b,ismatch3b,Valence3b,ACC4b,ismatch4b,Valence4b
0,0.002521,-0.008336,0.000020,0.050461,-0.000683,0.008252,0.000002,0.052448,0.000575,0.012569,0.000669,0.029183,0.001166,-0.007103,0.000656,0.007915
1,0.002313,0.003921,-0.000343,0.025837,0.000418,-0.002200,-0.000814,-0.013786,0.001072,0.006446,0.000469,0.034901,-0.000101,-0.001363,-0.000427,0.012584
2,0.000361,0.007155,0.000531,0.045688,-0.000435,0.003362,0.000311,0.036107,0.000172,0.000075,0.000076,0.013518,-0.000480,0.004416,-0.000321,0.022288
3,-0.000269,0.010052,0.000062,0.018004,0.000547,-0.003170,-0.000640,0.045182,-0.000829,0.003391,-0.000492,0.025354,-0.001972,0.006928,-0.000854,0.026549
4,0.000044,0.007343,0.000383,0.018577,0.001149,0.003636,-0.000059,0.022987,0.000629,0.001661,0.000528,0.001962,0.000621,0.008728,-0.000447,-0.000113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,0.000068,0.001751,0.000201,0.012652,-0.000207,-0.000487,-0.000844,0.019896,-0.000412,-0.003249,0.000270,0.015580,0.000300,0.003666,-0.000831,0.013760
160,0.000749,0.005556,-0.000163,0.016401,0.000128,0.002176,0.000015,0.026652,0.000088,-0.000684,-0.000163,0.010488,-0.000545,-0.007451,0.000419,0.016722
161,0.000114,0.003212,0.000209,0.002986,0.000265,0.001132,-0.000695,0.012450,0.000021,0.000795,0.000142,0.018926,-0.000214,0.003919,0.000077,0.016142
162,0.000168,-0.002092,0.000083,0.014737,0.000017,0.002266,0.000025,0.009134,-0.000294,0.000344,-0.000092,0.014342,0.000576,-0.001745,-0.000360,0.014088


In [19]:
#No ref to self ref trial back
df_cross1, im_cross1, coef_cross1 = lr_cross_task(X_source=X_norefb, y_source=y_noref, X_target=X_selfrefb, y_target=y_selfref, target_group=groups_self, source_name="No_Ref", target_name="Self_Ref")
#self to no ref trial back
df_cross2, im_cross2, coef_cross2 = lr_cross_task(X_source=X_selfrefb, y_source=y_selfref, X_target=X_norefb, y_target=y_noref, target_group=groups_no, source_name="Self_Ref", target_name="No_Ref")

In [20]:
df_score = pd.concat([score_no, score_self])

In [21]:
df_score.to_csv("LR_withinscore.csv")

In [22]:
df_cross_score = pd.concat([df_cross1, df_cross2])

In [23]:
df_cross_score.to_csv("LR_crossscore.csv")

In [24]:
df_im_no = pd.DataFrame(im_no, columns=norefcolb)
df_im_no["subj_idx"] = np.arange(1, len(im_no)+1)
df_im_no["source"] = "No_Ref"
df_im_no["target"] = "No_Ref"

df_im_self = pd.DataFrame(im_self, columns=selfrefcolb)
df_im_self["subj_idx"] = np.arange(1, len(im_self)+1)
df_im_self["source"] = "Self_Ref"
df_im_self["target"] = "Self_Ref"

In [25]:
im_no_long = pd.melt(df_im_no, id_vars=["subj_idx", "source", "target"])
im_self_long = pd.melt(df_im_self, id_vars=["subj_idx", "source", "target"])
im_within = pd.concat([im_no_long, im_self_long])
im_within

,subj_idx,source,target,variable,value
0,1,No_Ref,No_Ref,RT1b,0.002521
1,2,No_Ref,No_Ref,RT1b,0.002313
2,3,No_Ref,No_Ref,RT1b,0.000361
3,4,No_Ref,No_Ref,RT1b,-0.000269
4,5,No_Ref,No_Ref,RT1b,0.000044
...,...,...,...,...,...
3115,191,Self_Ref,Self_Ref,Valence4b,0.107084
3116,192,Self_Ref,Self_Ref,Valence4b,0.127071
3117,193,Self_Ref,Self_Ref,Valence4b,0.114251
3118,194,Self_Ref,Self_Ref,Valence4b,0.116026


In [26]:
im_within.to_csv("importance_within.csv")

In [27]:
df_im_cross1 = pd.DataFrame(im_cross1, columns=selfrefcolb)
df_im_cross1["subj_idx"] = np.arange(1, len(im_cross1)+1)
df_im_cross1["source"] = "No_Ref"
df_im_cross1["target"] = "Self_Ref"

df_im_cross2 = pd.DataFrame(im_cross2, columns=norefcolb)
df_im_cross2["subj_idx"] = np.arange(1, len(im_cross2)+1)
df_im_cross2["source"] = "Self_Ref"
df_im_cross2["target"] = "No_Ref"

In [28]:
im_cross1_long = pd.melt(df_im_cross1, id_vars=["subj_idx", "source", "target"])
im_cross2_long = pd.melt(df_im_cross2, id_vars=["subj_idx", "source", "target"])
im_cross = pd.concat([im_cross1_long, im_cross2_long])
im_cross

,subj_idx,source,target,variable,value
0,1,No_Ref,Self_Ref,RT1b,0.001067
1,2,No_Ref,Self_Ref,RT1b,-0.000395
2,3,No_Ref,Self_Ref,RT1b,0.000222
3,4,No_Ref,Self_Ref,RT1b,0.000561
4,5,No_Ref,Self_Ref,RT1b,0.000175
...,...,...,...,...,...
2619,160,Self_Ref,No_Ref,Valence4b,0.012466
2620,161,Self_Ref,No_Ref,Valence4b,0.015386
2621,162,Self_Ref,No_Ref,Valence4b,0.021403
2622,163,Self_Ref,No_Ref,Valence4b,0.019163


In [29]:
im_cross.to_csv("importance_cross.csv")

In [30]:
np.save('coef_no.npy', coef_no)
np.save('coef_self.npy', coef_self)
np.save('coef_cross1.npy', coef_cross1)
np.save('coef_cross2.npy', coef_cross2)

In [19]:
ct = ColumnTransformer(transformers=[("rt", StandardScaler(), [1, 5, 9, 13])],
                       remainder='passthrough')
logi = make_pipeline(
            ct, 
            LogisticRegressionCV(Cs = np.logspace(-6, 3, 7), cv = 5, class_weight='balanced', 
                                 random_state=123, max_iter=5000, multi_class="ovr"))



logi.fit(X=X_norefb, y=y_noref)
logi.predict_proba(X_selfrefb)

array([[0.33429728, 0.33331387, 0.33238885],
       [0.3341523 , 0.3333484 , 0.3324993 ],
       [0.33421859, 0.33336483, 0.33241657],
       ...,
       [0.333904  , 0.33333096, 0.33276504],
       [0.33351447, 0.33320205, 0.33328347],
       [0.33452359, 0.33327122, 0.33220519]])